In [1]:
# imports
import torch
from torch.utils.data import DataLoader

In [2]:
# unpickle function -> from https://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
# some variables
DATA_FOLDER = './data/'
C_FOLDER = 'cifar-10-batches-py/'
nn_batches_amount = 64

In [4]:
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
urlopen = urllib.request.urlopen('https://www.cs.toronto.edu/~kriz/cifar.html')

In [5]:
from torchvision import datasets, transforms

# getting the training and testing data
training_data = datasets.CIFAR10(DATA_FOLDER, download=True, train=True, transform=transforms.ToTensor())
testing_data = datasets.CIFAR10(DATA_FOLDER, download=False, train=False, transform=transforms.ToTensor())

# unpickling the text labels
batches_labels = unpickle(DATA_FOLDER + C_FOLDER + 'batches.meta')[b'label_names']

Files already downloaded and verified


In [6]:
# data loaders
training_set = DataLoader(training_data, batch_size=nn_batches_amount, shuffle=True)
testing_set = DataLoader(testing_data, batch_size=nn_batches_amount, shuffle=False)

In [7]:
# get label name
def get_label(i):
    return str(batches_labels[i])

# visualize images -> given a tensor of shape 3072 | 32 x 32 x 3
import matplotlib.pyplot as plt
def visual_img(img):
    img = img.view(3,32,32)
    img = img.swapaxes(0,1)
    img = img.swapaxes(1,2)
    plt.imshow(img)
    plt.show()

In [8]:
# the neural network
import torch.nn as nn
from torch.nn.functional import relu, log_softmax, nll_loss
import torch.optim as optim

# the network
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        self.hl1 = nn.Linear(32*32*3, 128)
        self.hl2 = nn.Linear(128, 128)
        self.hl3 = nn.Linear(128, 128)
        self.hl4 = nn.Linear(128, 64)
        self.hls = nn.Linear(64, 64)
        self.hl5 = nn.Linear(64, 10)

    def forward(self, target):
        target = relu(self.hl1(target))
        target = relu(self.hl2(target))
        target = relu(self.hl3(target))
        target = relu(self.hl4(target))
        target = torch.sigmoid(self.hls(target))
        target = log_softmax(self.hl5(target), dim=1)
        return target

# the network
network = Network()

# loss algorithm & optimizer
loss_algo = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

NameError: name 'device' is not defined

In [ ]:
# testing the neural network using the testing set
def test():
    predictions = 0

    with torch.no_grad():
        for batch in testing_set:
            images, targets = batch
            output = network(images.view(-1,32*32*3))
            
            for key, value in enumerate(output):
                prediction = torch.argmax(value)
                actual = targets[key]
                if prediction == actual:
                    predictions += 1
    return round(predictions/10000*100, 3)

In [ ]:
from tqdm import tqdm

# training the neural network
accuracy = []
for gen in tqdm(range(10)):
    for batch in training_set:

        # getting the images and labels from each batch
        images, target = batch

        # resetting the gradient to zero
        optimizer.zero_grad()

        # inputting the images into the network
        output = network(images.view(-1,32*32*3))
        
        # backward
        loss_algo = nll_loss(output, target)
        loss_algo.backward()

        # forward
        optimizer.step()

In [ ]:
test()

In [ ]:
from torch.nn.functional import max_pool2d

# the network
class ImprovedNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        
        # convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # interconnected
        self.l1 = nn.Linear(16 * 5 * 5, 128)
        self.l2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 10)

    def forward(self, target):
      target = max_pool2d(relu(self.conv1(target)), (2,2))
      target = max_pool2d(relu(self.conv2(target)), (2,2))

      target = torch.flatten(target, 1)
      target = relu(self.l1(target))
      target = relu(self.l2(target))
      target = log_softmax(self.out(target), dim=1)
      return target

# the network
improved_net = ImprovedNetwork()

# loss algorithm & optimizer
loss_algo = nn.CrossEntropyLoss()
optimizer = optim.Adam(improved_net.parameters(), lr=0.001)

Training

In [ ]:
# training the neural network
for gen in tqdm(range(3)):
    for batch in training_set:

        # getting the images and labels from each batch
        images, target = batch

        # resetting the gradient to zero
        optimizer.zero_grad()

        # inputting the images into the network
        output = improved_net(images.view(-1, 1, 32, 32))
        
        # backward
        loss_algo = nll_loss(output, target)
        loss_algo.backward()

        # forward
        optimizer.step()

In [ ]:
predictions = 0

with torch.no_grad():
    for batch in testing_set:
        images, targets = batch
        output = improved_net(images.view(-1, 3, 32, 32))
        
        for key, value in enumerate(output):
            prediction = torch.argmax(value)
            actual = targets[key]
            if prediction == actual:
                predictions += 1
                
print(round(predictions/10000*100, 3))